In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=4dbf2efba6088899d01d38ddc13d185773b3ab4a2d600dcb73bf2d14c771bf71
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [ ]:
import os
os.chdir("/content/drive/Shareddrives/contest/")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pchome = pd.read_csv('data/2021datapilot_PChome.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
pchome.rename(columns={'Unnamed: 0':'MEM_ID', 'Unnamed: 1':'POSTAL_CD','Unnamed: 2':'ORDER_ID'}, inplace = True)
pchome['DATE_CD'] = pd.to_datetime(pchome['DATE_CD'])
today=pd.to_datetime('2020-12-31')
pchome['PRICE'] = pchome['PRICE'].str.replace(',','').str.replace('$','').astype('int')

In [ ]:
class_mapping = { '否':0, '是':1}
pchome[ 'PRIME' ] = pchome[ 'PRIME' ].map(class_mapping)

In [ ]:
abbr=[]
for pg_id in pchome['PROD_ID']:
  abbr.append(pg_id[:4])

In [ ]:
pchome['PROD_abbr']=abbr
pchome.head()

,MEM_ID,POSTAL_CD,ORDER_ID,DATE_CD,TIME_CD,DEPARTMENT,GOODS,PROD_ID,PRICE,PRIME,PROD_abbr
0,c87fdf5e-9dca-4a3e-e053-6f1f0a0ac0f8,401,20200101000924,2020-01-01,12:59:22,家電,【一次換到好】2入 Coway：AP-1512HH 旗艦環禦型 水洗濾網(取代活性碳&HEP...,DMAU0C-A900AETV8-000,2693,0,DMAU
1,c87fdf5e-9dca-4a3e-e053-6f1f0a0ac0f8,401,20200101000924,2020-01-01,12:59:22,家電,【怡悅HEPA濾心】適用honeywell HPA-100APTW/HPA-200APTW/...,DMAU0D-A90054ZZQ-000,1140,0,DMAU
2,c87fdf5e-9dca-4a3e-e053-6f1f0a0ac0f8,401,20200101000924,2020-01-01,12:59:22,家電,【怡悅HEPA濾心】適用honeywell HPA-100APTW/HPA-200APTW/...,DMAU0D-A90054ZZQ-000,1140,0,DMAU
3,c87fdf5e-9dca-4a3e-e053-6f1f0a0ac0f8,401,20200113182398,2020-01-13,11:33:43,休閒,Durex 杜蕾斯特級潤滑液,DEAJ17-A42975500-000,310,0,DEAJ
4,c87fdf5e-9dca-4a3e-e053-6f1f0a0ac0f8,401,20200113182398,2020-01-13,11:33:43,休閒,DUREX杜蕾斯 超潤滑型 保險套 12入,DABC1S-A9005YCU6-000,300,0,DABC


# 衛生紙

In [ ]:
sub ='衛生紙'
order=np.unique(pchome.iloc[np.where(pchome.GOODS.str.find(sub)!=-1)].ORDER_ID)
tissue=pchome[pchome['ORDER_ID'].isin(order)]

In [ ]:
tissue.head()

,MEM_ID,POSTAL_CD,ORDER_ID,DATE_CD,TIME_CD,DEPARTMENT,GOODS,PROD_ID,PRICE,PRIME,PROD_abbr
33,c87fdf5e-9dca-4a3e-e053-6f1f0a0ac0f8,401,20201020954973,2020-10-20,15:43:11,日用,歐樂B-兒童牙刷(5-7歲)公主,DAAL07-A9008O0WF-000,49,0,DAAL
34,c87fdf5e-9dca-4a3e-e053-6f1f0a0ac0f8,401,20201020954973,2020-10-20,15:43:11,日用,歐樂B-兒童牙刷(2-4歲),DAAL07-A9008O0VR-000,49,0,DAAL
35,c87fdf5e-9dca-4a3e-e053-6f1f0a0ac0f8,401,20201020954973,2020-10-20,15:43:11,食品,【壽滿趣】紐西蘭Mother Earth超級絲滑花生醬380g,DBAAD7-A9008PHTF-000,259,0,DBAA
36,c87fdf5e-9dca-4a3e-e053-6f1f0a0ac0f8,401,20201020954973,2020-10-20,15:43:11,生活,角落小夥伴可愛不鏽鋼筷子-粉-52441A,DECU1L-A900AGUQ4-000,69,0,DECU
37,c87fdf5e-9dca-4a3e-e053-6f1f0a0ac0f8,401,20201020954973,2020-10-20,15:43:11,日用,舒潔 濕式衛生紙 (40片x16包),DAAG4S-A90081NB4-000,649,0,DAAG


In [ ]:
def merge(array):
  return(list(array))
product=tissue.groupby(['MEM_ID','ORDER_ID']).agg({'PROD_abbr':merge})

In [ ]:
product

PROD_abbr
MEM_ID                               ORDER_ID                                                         
c87fdf5e-9dca-4a3e-e053-6f1f0a0ac0f8 20201020954973   [DAAL, DAAL, DBAA, DECU, DAAG, DECU, DAAJ, DBAJ]
                                     20201221348063   [DAAN, DAAN, DAAJ, DAAN, DEAZ, DAAN, DECT, DEAC]
c87fdf5e-9dcb-4a3e-e053-6f1f0a0ac0f8 20200523972260                                       [DXAB, DAAG]
                                     20200808430331                           [DAAG, DRAH, DSAB, DAAG]
                                     20201219085092                                             [DAAG]
...                                                                                                ...
c87fdf5f-5b54-4a3e-e053-6f1f0a0ac0f8 20200421470995                                 [DBAS, DAAG, DBCO]
                                     20201231817752  [DEBA, QFAD, QFAD, DEBA, DEBA, DEAZ, DEAB, DEB...
c87fdf5f-5b56-4a3e-e053-6f1f0a0ac0f8 20200104405417                                       [DAAK, DAAG]
                                     20200507584746                                             [DAAG]
                                     20200823110037                                 [DAAK, DAAK, DAAG]

[21756 rows x 1 columns]

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
te = TransactionEncoder()
#進行 one-hot 編碼
te_ary = te.fit(list(product['PROD_abbr'])).transform(list(product['PROD_abbr']))
df = pd.DataFrame(te_ary, columns=te.columns_)
#利用 Apriori 找出頻繁項集
freq = apriori(df, min_support=0.01, use_colnames=True)

#匯入關聯規則包
from mlxtend.frequent_patterns import association_rules
#計算關聯規則
result = association_rules(freq, metric="confidence", min_threshold=0.6)

支持度（Support）：支持度可以理解为物品当前流行程度。：

支持度 = （包含物品A的记录数量） / （总的记录数量）  
用上面的超市记录举例，一共有五个交易，牛奶出现在三个交易中，故而{牛奶}的支持度为3/5。{鸡蛋}的支持度是4/5。牛奶和鸡蛋同时出现的次数是2，故而{牛奶，鸡蛋}的支持度为2/5  

置信度（Confidence）：置信度是指如果购买物品A，有较大可能购买物品B。计算方式是这样：

置信度( A -> B) = （包含物品A和B的记录数量） / （包含 A 的记录数量）  
例：我们已经知道，(牛奶，鸡蛋)一起购买的次数是两次，鸡蛋的购买次数是4次。那么Confidence(牛奶->鸡蛋)的计算方式是Confidence(牛奶->鸡蛋)=2 / 4  

提升度（Lift）：提升度指当销售一个物品时，另一个物品销售率会增加多少。计算方式是：

提升度( A -> B) = 置信度( A -> B) / (支持度 A)  
当提升度(A->B)的值大于1的时候，说明物品A卖得越多，B也会卖得越多。而提升度等于1则意味着产品A和B之间没有关联。最后，提升度小于1那么意味着购买A反而会减少B的销量。


In [ ]:
word=['洗衣精','牙膏','尿布','醫療用品','主頁', '飲料','零食','進口零食','生機','礦泉水','沖泡', '罐頭','進口飲料','進口食材','速食麵','米/食材','零食/進口零食']

In [ ]:
df=result.iloc[np.where(result['lift']>1)]
df['object']=word
t_df=df.loc[:,['object','support','confidence','lift']]
t_df1=t_df.iloc[np.where(t_df['support']>0.02)]
t_df2=t_df1.iloc[np.where(t_df1['confidence']>0.98)]
t_df2.to_csv('tissue_association.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# 奶茶

In [ ]:
sub ='奶茶'
order=np.unique(pchome.iloc[np.where(pchome.GOODS.str.find(sub)!=-1)].ORDER_ID)
milk_tea=pchome[pchome['ORDER_ID'].isin(order)]

In [ ]:
def merge(array):
  return(list(array))
product=milk_tea.groupby(['MEM_ID','ORDER_ID']).agg({'PROD_abbr':merge})

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
te = TransactionEncoder()
#進行 one-hot 編碼
te_ary = te.fit(list(product['PROD_abbr'])).transform(list(product['PROD_abbr']))
df = pd.DataFrame(te_ary, columns=te.columns_)
#利用 Apriori 找出頻繁項集
freq = apriori(df, min_support=0.01, use_colnames=True)

#匯入關聯規則包
from mlxtend.frequent_patterns import association_rules
#計算關聯規則
result = association_rules(freq, metric="confidence", min_threshold=0.6)

In [ ]:
result

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(DBAA),(DBAB),0.145273,0.568025,0.106072,0.730159,1.285435,0.023554,1.600850
1,(DBAA),(DBAC),0.145273,0.328978,0.094927,0.653439,1.986272,0.047135,1.936232
2,(DBAC),(DBAB),0.328978,0.568025,0.225211,0.684579,1.205193,0.038344,1.369522
3,(DBAE),(DBAB),0.184089,0.568025,0.116833,0.634656,1.117303,0.012266,1.182378
4,(DBAF),(DBAB),0.021138,0.568025,0.013451,0.636364,1.120310,0.001445,1.187932
...,...,...,...,...,...,...,...,...,...
528,"(DBAC, DBAY, DBBM, DBAE)",(DBAB),0.013836,0.568025,0.012298,0.888889,1.564877,0.004439,3.887779
529,"(DBAB, DBAY, DBBM)","(DBAC, DBAE)",0.015373,0.140277,0.012298,0.800000,5.703014,0.010142,4.298616
530,"(DBAY, DBBM, DBAE)","(DBAB, DBAC)",0.015373,0.225211,0.012298,0.800000,3.552218,0.008836,3.873943
531,"(DBAC, DBAY, DBBM)","(DBAB, DBAE)",0.016910,0.116833,0.012298,0.727273,6.224880,0.010323,3.238278


# 紙尿褲

In [ ]:
sub ='紙尿褲'
order=np.unique(pchome.iloc[np.where(pchome.GOODS.str.find(sub)!=-1)].ORDER_ID)
Diapers=pchome[pchome['ORDER_ID'].isin(order)]

In [ ]:
def merge(array):
  return(list(array))
product=Diapers.groupby(['MEM_ID','ORDER_ID']).agg({'PROD_abbr':merge})

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
te = TransactionEncoder()
#進行 one-hot 編碼
te_ary = te.fit(list(product['PROD_abbr'])).transform(list(product['PROD_abbr']))
df = pd.DataFrame(te_ary, columns=te.columns_)
#利用 Apriori 找出頻繁項集
freq = apriori(df, min_support=0.01, use_colnames=True)

#匯入關聯規則包
from mlxtend.frequent_patterns import association_rules
#計算關聯規則
result = association_rules(freq, metric="confidence", min_threshold=0.6)

In [ ]:
rowname= ['掃除用具', '3M', '零食', '婦幼', '嬰童']
result2=result.iloc[np.where(result['lift']>1)]
result2['object']=rowname
result2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,object
1,(DAAC),(DAAO),0.012841,0.889534,0.011609,0.904110,1.016386,0.000187,1.152004,掃除用具
6,(DAAN),(DAAO),0.017414,0.889534,0.016007,0.919192,1.033341,0.000516,1.367018,3M
10,(DBAC),(DAAO),0.012489,0.889534,0.011609,0.929577,1.045016,0.000500,1.568619,零食
11,(DEAI),(DAAO),0.041161,0.889534,0.040809,0.991453,1.114576,0.004195,12.924538,婦幼
12,(DECH),(DAAO),0.011609,0.889534,0.011258,0.969697,1.090118,0.000931,3.645383,嬰童


# 滑鼠

In [ ]:
sub ='滑鼠'
order=np.unique(pchome.iloc[np.where(pchome.GOODS.str.find(sub)!=-1)].ORDER_ID)
Diapers=pchome[pchome['ORDER_ID'].isin(order)]

def merge(array):
  return(list(array))
product=Diapers.groupby(['MEM_ID','ORDER_ID']).agg({'PROD_abbr':merge})

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
te = TransactionEncoder()
#進行 one-hot 編碼
te_ary = te.fit(list(product['PROD_abbr'])).transform(list(product['PROD_abbr']))
df = pd.DataFrame(te_ary, columns=te.columns_)
#利用 Apriori 找出頻繁項集
freq = apriori(df, min_support=0.01, use_colnames=True)

#匯入關聯規則包
from mlxtend.frequent_patterns import association_rules
#計算關聯規則
result = association_rules(freq, metric="confidence", min_threshold=0.6)

In [ ]:
rowname=['洗衣精', '零食', '進口零食', '隨身碟']
df=result.iloc[np.intersect1d(np.where(result['lift']>1), np.where(result['confidence']>0.75))]
df['object']=rowname
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,object
2,(DAAK),(DCAN),0.014425,0.712714,0.011491,0.796610,1.117714,0.001210,1.412490,洗衣精
6,(DBAC),(DCAN),0.026650,0.712714,0.021027,0.788991,1.107023,0.002033,1.361486,零食
7,(DBAE),(DCAN),0.017359,0.712714,0.013692,0.788732,1.106661,0.001320,1.359821,進口零食
12,(DGCD),(DCAN),0.014181,0.712714,0.010758,0.758621,1.064411,0.000651,1.190185,隨身碟


# 3M

In [ ]:
sub ='餐具'
order=np.unique(pchome.iloc[np.where(pchome.GOODS.str.find(sub)!=-1)].ORDER_ID)
Diapers=pchome[pchome['ORDER_ID'].isin(order)]

def merge(array):
  return(list(array))
product=Diapers.groupby(['MEM_ID','ORDER_ID']).agg({'PROD_abbr':merge})

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
te = TransactionEncoder()
#進行 one-hot 編碼
te_ary = te.fit(list(product['PROD_abbr'])).transform(list(product['PROD_abbr']))
df = pd.DataFrame(te_ary, columns=te.columns_)
#利用 Apriori 找出頻繁項集
freq = apriori(df, min_support=0.01, use_colnames=True)

#匯入關聯規則包
from mlxtend.frequent_patterns import association_rules
#計算關聯規則
result = association_rules(freq, metric="confidence", min_threshold=0.6)

In [ ]:
rowname=['衛生紙、掃除用具', '沐浴乳、掃除用具','洗衣精、掃除用具','家用清潔劑、掃除用具','進口食材、掃除用具','衛浴、掃除用具',
         '衛生紙、沐浴乳', '衛生紙、洗衣精','衛生紙、家用清潔劑','家用清潔劑、洗衣精', '衛生紙、家用清潔劑、掃除用具',
         '家用清潔劑、洗衣精、掃除用具','餐廚、家用清潔劑、掃除用具']
df=result.iloc[np.intersect1d(np.where(result['lift']>1.7), np.where(result['confidence']>0.75))]
df['object']=rowname
print('3M')
df

3M


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,object
22,"(DAAG, DAAC)",(DAAN),0.035836,0.485779,0.030717,0.857143,1.764470,0.013308,3.599545,衛生紙、掃除用具
23,"(DAAJ, DAAC)",(DAAN),0.022753,0.485779,0.019909,0.875000,1.801230,0.008856,4.113766,沐浴乳、掃除用具
24,"(DAAK, DAAC)",(DAAN),0.025028,0.485779,0.022753,0.909091,1.871407,0.010595,5.656428,洗衣精、掃除用具
26,"(DAAZ, DAAC)",(DAAN),0.055745,0.485779,0.047782,0.857143,1.764470,0.020702,3.599545,家用清潔劑、掃除用具
27,"(DBAK, DAAC)",(DAAN),0.012514,0.485779,0.010808,0.863636,1.777837,0.004729,3.770952,進口食材、掃除用具
29,"(DECB, DAAC)",(DAAN),0.015358,0.485779,0.013083,0.851852,1.753578,0.005622,3.470990,衛浴、掃除用具
30,"(DAAG, DAAJ)",(DAAN),0.012514,0.485779,0.010808,0.863636,1.777837,0.004729,3.770952,衛生紙、沐浴乳
31,"(DAAG, DAAK)",(DAAN),0.015927,0.485779,0.014221,0.892857,1.837989,0.006484,4.799393,衛生紙、洗衣精
32,"(DAAG, DAAZ)",(DAAN),0.031854,0.485779,0.026735,0.839286,1.727710,0.011261,3.199595,衛生紙、家用清潔劑
35,"(DAAZ, DAAK)",(DAAN),0.030717,0.485779,0.025597,0.833333,1.715457,0.010676,3.085324,家用清潔劑、洗衣精


# 網路線 (線材)

In [ ]:
sub ='網路線'
order=np.unique(pchome.iloc[np.where(pchome.GOODS.str.find(sub)!=-1)].ORDER_ID)
Diapers=pchome[pchome['ORDER_ID'].isin(order)]

def merge(array):
  return(list(array))
product=Diapers.groupby(['MEM_ID','ORDER_ID']).agg({'PROD_abbr':merge})

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
te = TransactionEncoder()
#進行 one-hot 編碼
te_ary = te.fit(list(product['PROD_abbr'])).transform(list(product['PROD_abbr']))
df = pd.DataFrame(te_ary, columns=te.columns_)
#利用 Apriori 找出頻繁項集
freq = apriori(df, min_support=0.01, use_colnames=True)

#匯入關聯規則包
from mlxtend.frequent_patterns import association_rules
#計算關聯規則
result = association_rules(freq, metric="confidence", min_threshold=0.6)

In [ ]:
result

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(DAAC),(DCAC),0.021834,0.901747,0.018559,0.850000,0.942615,-0.001130,0.655022
1,(DAAG),(DCAC),0.042576,0.901747,0.040393,0.948718,1.052089,0.002000,1.915939
2,(DAAN),(DCAC),0.033843,0.901747,0.029476,0.870968,0.965867,-0.001042,0.761463
3,(DAAZ),(DCAC),0.024017,0.901747,0.022926,0.954545,1.058552,0.001268,2.161572
4,(DBAB),(DCAC),0.012009,0.901747,0.010917,0.909091,1.008144,0.000088,1.080786
5,(DBAC),(DCAC),0.017467,0.901747,0.016376,0.937500,1.039649,0.000625,1.572052
6,(DCAB),(DCAC),0.021834,0.901747,0.019651,0.900000,0.998063,-0.000038,0.982533
7,(DCAD),(DCAC),0.034934,0.901747,0.031659,0.906250,1.004994,0.000157,1.048035
8,(DCAH),(DCAC),0.021834,0.901747,0.019651,0.900000,0.998063,-0.000038,0.982533
9,(DCAN),(DCAC),0.017467,0.901747,0.016376,0.937500,1.039649,0.000625,1.572052


In [ ]:
rowname=['文具', '衛浴','記憶卡','隨身碟','TP-Link旗鑑館','LCD 螢幕','官網','手機／平板配件']
df=result.iloc[np.intersect1d(np.where(result['lift']>1.1), np.where(result['confidence']>0.95))]
df['object']=rowname
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,object
12,(DEAH),(DCAC),0.013100,0.901747,0.013100,1.0,1.108959,0.001287,inf,文具
14,(DECB),(DCAC),0.016376,0.901747,0.016376,1.0,1.108959,0.001609,inf,衛浴
15,(DGAG),(DCAC),0.016376,0.901747,0.016376,1.0,1.108959,0.001609,inf,記憶卡
17,(DGCD),(DCAC),0.013100,0.901747,0.013100,1.0,1.108959,0.001287,inf,隨身碟
20,(DRAN),(DCAC),0.012009,0.901747,0.012009,1.0,1.108959,0.001180,inf,TP-Link旗鑑館
21,(DSAB),(DCAC),0.030568,0.901747,0.030568,1.0,1.108959,0.003003,inf,LCD 螢幕
23,(DSBA),(DCAC),0.014192,0.901747,0.014192,1.0,1.108959,0.001394,inf,官網
24,(DYAF),(DCAC),0.012009,0.901747,0.012009,1.0,1.108959,0.001180,inf,手機／平板配件
